In [ ]:
from pathlib import Path
import numpy as np
import torch
from tirex import ForecastModel, load_model
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from typing import Dict
import matplotlib.dates as mdates

model: ForecastModel = load_model("tirexmodel.ckpt")

project_path  = os.environ.get('TMPDIR') + '/wissdaten/ZKI-PH4/sprint_mosqlimate/data/processed'
admin_level = 'state'
region_col = 'uf'
q10 = []
q20 = []
q30 = []
q40 = []
q50 = []
q60 = []
q70 = []
q80 = []
q90 = []
pred_list = []
state_list = []
val_set_list = []
dates_list = []
for val_set in [1, 2, 3]:
    context_path = os.path.join(project_path, f'train_std_{admin_level}_{val_set}.csv')
    context = pd.read_csv(context_path)[['date', region_col, 'cases']]

    dates_path = os.path.join(project_path, f'val_std_{admin_level}_{val_set}.csv')
    dates = pd.read_csv(dates_path)[['date']]
    dates = dates.date.unique().tolist()
    prediction_length = len(dates)

    for state in context[region_col].unique():
        
        context_state = context.loc[context[region_col] == state].copy()
        pred_raw, preds_ = model.forecast(
                    context=context_state['cases'].values,
                    prediction_length=prediction_length
                )
        val_set_list+=[val_set]*prediction_length
        dates_list+=dates
        state_list+=[state]*prediction_length
        pred_list+=preds_[0].tolist()
        q10+=pred_raw[0,:,0].tolist()
        q20+=pred_raw[0,:,1].tolist()
        q30+=pred_raw[0,:,2].tolist()
        q40+=pred_raw[0,:,3].tolist()
        q50+=pred_raw[0,:,4].tolist()
        q60+=pred_raw[0,:,5].tolist()
        q70+=pred_raw[0,:,6].tolist()
        q80+=pred_raw[0,:,7].tolist()
        q90+=pred_raw[0,:,8].tolist()

forecast_results = pd.DataFrame({'val_set_id': val_set_list, 'date': dates_list, 'uf': state_list, 'cases_pred': pred_list, 
                                 'q10': q10, 'q20': q20, 'q30': q30, 'q40': q40, 'q50': q50,
                                 'q60': q60, 'q70': q70, 'q80': q80, 'q90': q90,})
forecast_results.to_csv(os.environ.get('TMPDIR') + '/wissdaten/ZKI-PH4/sprint_mosqlimate/data/forecasts/tirex/context=all_cases.csv', sep=',', index=False, header=True)

Loading weights from local directory


/home/bangertb/.conda/envs/tirex/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/home/bangertb/.conda/envs/tirex/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(


1
2
3


In [6]:
import pandas as pd
project_path  = os.environ.get('TMPDIR') + '/wissdaten/ZKI-PH4/sprint_mosqlimate/data/processed'
context_path = os.path.join(project_path, f'train_std_state_1.csv')
context = pd.read_csv(context_path).columns

In [22]:
from pathlib import Path
import numpy as np
import torch
from tirex import ForecastModel, load_model
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pandas as pd
from typing import Dict
import matplotlib.dates as mdates
from sklearn.metrics import root_mean_squared_error
model: ForecastModel = load_model("NX-AI/TiRex/model.ckpt")

#!hpc-ticket
#!hpc-mount wissdaten

project_path  = os.environ.get('TMPDIR') + '/wissdaten/ZKI-PH4/sprint_mosqlimate/data/processed'
admin_level = 'state'
region_col = 'uf'
columns_to_forecast = ['cases',
       'temp_min', 'temp_med', 'temp_max', 'precip_min', 'precip_med',
       'precip_max', 'pressure_min', 'pressure_med', 'pressure_max',
       'rel_humid_min', 'rel_humid_med', 'rel_humid_max', 'thermal_range',
       'rainy_days', 'enso', 'iod', 'pdo']

for feature_col in columns_to_forecast:
    q10 = []
    q20 = []
    q30 = []
    q40 = []
    q50 = []
    q60 = []
    q70 = []
    q80 = []
    q90 = []
    pred_list = []
    state_list = []
    val_set_list = []
    dates_list = []
    rmse = 0
    for val_set in [1, 2, 3]:
        context_path = os.path.join(project_path, f'train_std_{admin_level}_{val_set}.csv')
        context = pd.read_csv(context_path)[['date', region_col, feature_col]]
    
        dates_path = os.path.join(project_path, f'val_std_{admin_level}_{val_set}.csv')
        val_data = pd.read_csv(dates_path)
        dates = val_data[['date']]
        dates = dates.date.unique().tolist()
        prediction_length = len(dates)
    
        for state in context[region_col].unique():
            
            context_state = context.loc[context[region_col] == state].copy()
            pred_raw, preds_ = model.forecast(
                        context=context_state[feature_col].values,
                        prediction_length=prediction_length
                    )
            val_set_list+=[val_set]*prediction_length
            dates_list+=dates
            state_list+=[state]*prediction_length
            pred_list+=preds_[0].tolist()
            q10+=pred_raw[0,:,0].tolist()
            q20+=pred_raw[0,:,1].tolist()
            q30+=pred_raw[0,:,2].tolist()
            q40+=pred_raw[0,:,3].tolist()
            q50+=pred_raw[0,:,4].tolist()
            q60+=pred_raw[0,:,5].tolist()
            q70+=pred_raw[0,:,6].tolist()
            q80+=pred_raw[0,:,7].tolist()
            q90+=pred_raw[0,:,8].tolist()
        
        rmse_df = pd.DataFrame({'date': dates_list[-prediction_length*len(context[region_col].unique()):], 'uf': state_list[-prediction_length*len(context[region_col].unique()):],
            f'{feature_col}_pred': pred_list[-prediction_length*len(context[region_col].unique()):]})
        rmse_df = pd.merge(rmse_df, val_data[['date', 'uf', feature_col]], on=['date', 'uf'], how='left')
        try:
            rmse+=root_mean_squared_error(rmse_df[feature_col],rmse_df[f'{feature_col}_pred'])
        except:
            rmse = 'NaN'
            print('NaN detected')
                
    forecast_results = pd.DataFrame({'val_set_id': val_set_list, 'date': dates_list, 'uf': state_list, feature_col: pred_list, 
                                     'q10': q10, 'q20': q20, 'q30': q30, 'q40': q40, 'q50': q50,
                                     'q60': q60, 'q70': q70, 'q80': q80, 'q90': q90,})
    if rmse == 'NaN':
        avgrmse = 'NaN'
    else:
        avgrmse = rmse / 3
    forecast_results.to_csv(os.environ.get('TMPDIR') + f'/wissdaten/ZKI-PH4/sprint_mosqlimate/data/forecasts/tirex/context=all_avgrmse={avgrmse}_{feature_col}.csv', sep=',', index=False, header=True)

Loading weights from local directory


/home/bangertb/.conda/envs/tirex/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:543: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @conditional_decorator(
/home/bangertb/.conda/envs/tirex/lib/python3.11/site-packages/xlstm/blocks/slstm/cell.py:568: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @conditional_decorator(


NaN detected
NaN detected
NaN detected
